## Issue - Hamiltoniain builder doesnt work for unrestricted

Overview:
- [] Force unrstricted on even number of electrons.
- [] Check for success on odd electrons.

Specifics:
The two body integrals are not being generated correctly. For one mixed spin term, only the alphas are in use, and for the others only the beta.
- [] Explicitly calculate the two body integrals for closed shell.
- [] Make sure this works for open shell.

In [93]:
"""Class to build qubit Hamiltonians from scf object."""
import logging
from typing import List, Optional, Tuple, Union

import numpy as np
import openfermion.transforms as of_transforms
from cached_property import cached_property
from openfermion import InteractionOperator, QubitOperator, count_qubits
from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion.config import EQ_TOLERANCE
from openfermion.ops.representations import get_active_space_integrals
from openfermion.transforms import taper_off_qubits
from pyscf import ao2mo, dft, scf
from pyscf.lib import StreamObject
from pyscf.lib.numpy_helper import SYMMETRIC
from qiskit.opflow import Z2Symmetries
from typing_extensions import final

from nbed.exceptions import HamiltonianBuilderError
from nbed.ham_converter import HamiltonianConverter

logger = logging.getLogger(__name__)


class HamiltonianBuilder:
    """Class to build molecular hamiltonians."""

    def __init__(
        self,
        scf_method: StreamObject,
        constant_e_shift: Optional[float] = 0,
        transform: Optional[str] = "jordan_wigner",
    ) -> None:
        """Initialise the HamiltonianBuilder.

        Args:
            scf_method: Pyscf scf object.
            constant_e_shift: Constant energy shift to apply to the Hamiltonian.
            transform: Transformation to apply to the Hamiltonian.
        """
        logger.debug("Initialising HamiltonianBuilder.")
        self.scf_method = scf_method
        self.constant_e_shift = constant_e_shift
        self.transform = transform

    @property
    def _one_body_integrals(self) -> np.ndarray:
        """Get the one electron integrals."""
        logger.debug("Calculating one body integrals.")
        c_matrix_active = self.scf_method.mo_coeff

        # one body terms
        if isinstance(self.scf_method, (scf.uhf.UHF, dft.uks.UKS)):
            logger.info("Calculating unrestricted one body intergrals.")
            one_body_integrals_alpha = (
                c_matrix_active[0].T
                @ self.scf_method.get_hcore()[0]
                @ c_matrix_active[0]
            )
            one_body_integrals_beta = (
                c_matrix_active[1].T
                @ self.scf_method.get_hcore()[1]
                @ c_matrix_active[1]
            )

            one_body_integrals = np.array(
                [one_body_integrals_alpha, one_body_integrals_beta]
            )

        else:
            logger.info("Calculating restricted one body integrals.")
            # We double these up so that we have the same number as 
            # the unrestricted case.
            one_body_integrals = np.array([
                c_matrix_active.T @ self.scf_method.get_hcore() @ c_matrix_active
            ] * 2)
        logger.debug("One body integrals found.")

        return one_body_integrals

    @property
    def _two_body_integrals(self) -> np.ndarray:
        """Get the two electron integrals."""
        logger.debug("Calculating two body integrals.")
        c_matrix_active = self.scf_method.mo_coeff

        if isinstance(self.scf_method, (scf.uhf.UHF, dft.uks.UKS)):
            n_orbs_alpha = c_matrix_active[0].shape[1]
            n_orbs_beta = c_matrix_active[1].shape[1]
            
            # Could make this more flexible later.
            if n_orbs_alpha != n_orbs_beta:
                raise HamiltonianBuilderError(
                    "Must localize the same number of alpha and beta orbitals."
                )

            c_alpha = c_matrix_active[0]
            c_beta = c_matrix_active[1]

            # Pyscf is in chemist notation
            # later we transpose to physicist notation for openfermion
            spin_options = {
                "aaaa": (c_alpha, c_alpha, c_alpha, c_alpha),
                "bbbb": (c_beta, c_beta, c_beta, c_beta),
                "aabb": (c_alpha, c_alpha, c_beta, c_beta),
                "bbaa": (c_beta, c_beta, c_alpha, c_alpha),
            }

            two_body_integrals = []
            for spin in spin_options:
                two_body_compressed = ao2mo.kernel(
                    self.scf_method.mol, spin_options[spin]
                )
                eri = ao2mo.restore(1, two_body_compressed, n_orbs_alpha)
                two_body_integrals.append(np.asarray(
                    eri.transpose(0, 2, 3, 1), order="C"
                ))


        else:
            n_orbs = c_matrix_active.shape[1]

            two_body_compressed = ao2mo.kernel(self.scf_method.mol, c_matrix_active)

            # get electron repulsion integrals
            eri = ao2mo.restore(
                1, two_body_compressed, n_orbs
            )  # no permutation symmetry

            # Copy this 4 times so that we have the same number as
            # the unrestricted case
            # Openfermion uses physicist notation whereas pyscf uses chemists
            two_body_integrals = [np.asarray(eri.transpose(0, 2, 3, 1), order="C")]*4

        logger.debug("Two body integrals found.")

        return two_body_integrals

    def _reduce_active_space(self, qubit_reduction: int) -> None:
        """Reduce the active space to accommodate a certain number of qubits."""
        logger.debug("Reducing the active space.")

        if type(qubit_reduction) is not int:
            logger.error("Invalid qubit_reduction of type %s.", type(qubit_reduction))
            raise HamiltonianBuilderError("qubit_reduction must be an Intger")
        if qubit_reduction == 0:
            logger.debug("No active space reduction required.")
            return 0, self._one_body_integrals, self._two_body_integrals

        # find where the last occupied level is
        scf = self.scf_method
        occupied = np.where(scf.mo_occ > 0)[0]
        unoccupied = np.where(scf.mo_occ == 0)[0]

        # +1 because each MO is 2 qubits for closed shell.
        n_orbitals = (qubit_reduction + 1) // 2
        logger.debug(f"Reducing to {n_orbitals}.")
        # Again +1 because we want to use odd numbers to reduce
        # occupied orbitals
        occupied_reduction = (n_orbitals + 1) // 2
        unoccupied_reduction = qubit_reduction - occupied_reduction

        # We want the MOs nearest the fermi level
        # unoccupied orbitals go from 0->N and occupied from N->M
        self._active_space_indices = np.append(
            occupied[occupied_reduction:], unoccupied[:unoccupied_reduction]
        )

        occupied_indices = np.where(self.scf_method.mo_occ > 0)[0]
        logger.debug(f"Active indices {self._active_space_indices}.")
        (
            core_constant,
            one_body_integrals,
            two_body_integrals,
        ) = get_active_space_integrals(
            self._one_body_integrals,
            self._two_body_integrals,
            occupied_indices=occupied_indices,
            active_indices=self._active_space_indices,
        )

        logger.debug("Active space reduced.")
        return core_constant, one_body_integrals, two_body_integrals

    @staticmethod
    def _qubit_transform(
        transform: str, intop: InteractionOperator
    ) -> QubitOperator:
        """Transform second quantised hamiltonain to qubit Hamiltonian.

        Args:
            transform: Transformation to apply to the Hamiltonian.
            intop: InteractionOperator to transform.

        Returns:
            QubitOperator: Transformed qubit Hamiltonian.
        """
        logger.debug(f"Transforming to qubit Hamiltonian using {transform} transform.")
        if transform is None or hasattr(of_transforms, transform) is False:
            raise HamiltonianBuilderError(
                "Invalid transform. Please use a transform from `openfermion.transforms`."
            )

        transform = getattr(of_transforms, transform)

        try:
            qubit_hamiltonain: QubitOperator = transform(intop)
        except TypeError:
            logger.error(
                "Transform selected is not a valid InteractionOperator transform."
            )
            raise HamiltonianBuilderError(
                "Transform selected is not a valid InteractionOperator transform."
            )

        if type(qubit_hamiltonain) is not QubitOperator:
            raise HamiltonianBuilderError(
                "Transform selected must output a QubitOperator."
            )

        logger.debug("Qubit Hamiltonian constructed.")
        return qubit_hamiltonain

    def _taper(self, qham: QubitOperator) -> QubitOperator:
        """Taper a hamiltonian.

        Args:
            qham: QubitOperator to taper.

        Returns:
            QubitOperator: Tapered QubitOperator.
        """
        logger.error("Tapering not implemented.")
        raise ValueError("tapering currently NOT working properly!")
        logger.debug("Beginning qubit tapering.")
        converter = HamiltonianConverter(qham)
        symmetries = Z2Symmetries.find_Z2_symmetries(converter.qiskit)
        symm_strings = [symm.to_label() for symm in symmetries.sq_paulis]

        logger.debug(f"Found {len(symm_strings)} Z2Symmetries")

        stabilizers = []
        for string in symm_strings:
            term = [
                f"{pauli}{index}" for index, pauli in enumerate(string) if pauli != "I"
            ]
            term = " ".join(term)
            stabilizers.append(QubitOperator(term=term))

        logger.debug("Tapering complete.")
        return taper_off_qubits(qham, stabilizers)

    def build(
        self, n_qubits: Optional[int] = None, taper: Optional[bool] = False
    ) -> QubitOperator:
        """Returns second quantized fermionic molecular Hamiltonian.

        constant_e_shift is a constant energy addition... in this code this will be the classical embedding energy
        that corrects for the full system.

        The active_indices and occupied indices are an active space approximation... where occupied and virtual orbitals
        can be frozen. This is different to removing the environment orbitals, as core_constant terms must be added to
        make this approximation.

        Args:
            scf_method (StreamObject): A pyscf self-consistent method.
            constant_e_shift (float): constant energy term to add to Hamiltonian
            active_indices (list): A list of spatial orbital indices indicating which orbitals should be
                                considered active.
            occupied_indices (list):  A list of spatial orbital indices indicating which orbitals should be
                                    considered doubly occupied.

        Returns:
            molecular_hamiltonian (InteractionOperator): fermionic molecular Hamiltonian
        """
        logger.debug("Building for %s qubits.", n_qubits)
        qubit_reduction = 0
        while True:
            (
                core_constant,
                one_body_integrals,
                two_body_integrals,
            ) = self._reduce_active_space(qubit_reduction)

            one_body_coefficients, two_body_coefficients = self._spinorb_from_spatial(
                one_body_integrals, two_body_integrals
            )

            molecular_hamiltonian = InteractionOperator(
                (self.constant_e_shift + core_constant),
                one_body_coefficients,
                0.5 * two_body_coefficients,
            )

            qham = self._qubit_transform(self.transform, molecular_hamiltonian)

            # Don't like this option sitting with the recursive
            # call beneath it - just a little too complicated.
            # ...but it works for now.
            if taper is True:
                qham = self._taper(qham)
            if n_qubits is None:
                logger.debug("Unreduced Hamiltonain found.")
                return qham

            # Wanted to do a recursive thing to get the correct number
            # from tapering but it takes ages.
            final_n_qubits = count_qubits(qham)
            if final_n_qubits <= n_qubits:
                logger.debug("Hamiltonian reduced to %s qubits.", final_n_qubits)
                return qham

            # Check that we have the right number of qubits.
            qubit_reduction += final_n_qubits - n_qubits

    
    def _spinorb_from_spatial(self, one_body_integrals, two_body_integrals) -> Tuple[np.ndarray, np.ndarray]:
        """Convert spatial integrals to spin-orbital integrals.
        Args:
            one_body_integrals (np.ndarray): One-electron integrals in physicist notation.
            two_body_integrals (np.ndarray): Two-electron integrals in physicist notation.

        Returns:
            one_body_coefficients (np.ndarray): One-electron coefficients in spinorb form.
            two_body_coefficients (np.ndarray): Two-electron coefficients in spinorb form.

        """

        n_qubits = one_body_integrals[0].shape[0] + one_body_integrals[1].shape[0]

        # Initialize Hamiltonian coefficients.
        one_body_coefficients = np.zeros((n_qubits, n_qubits))
        two_body_coefficients = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

        # Loop through integrals.
        for p in range(n_qubits // 2):
            for q in range(n_qubits // 2):

                # Populate 1-body coefficients. Require p and q have same spin.
                one_body_coefficients[2 * p, 2 * q] = one_body_integrals[0, p, q]
                # one_body_coefficients[2 * p, 2 * q] = one_body_integrals[1, p, q]

                # one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[0, p, q]
                one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[1, p, q]

                # Continue looping to prepare 2-body coefficients.
                for r in range(n_qubits // 2):
                    for s in range(n_qubits // 2):

                        # Same spin
                        two_body_coefficients[
                            2 * p, 2 * q, 2 * r, 2 * s
                        ] = two_body_integrals[0][p, q, r, s]
                        two_body_coefficients[
                            2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1
                        ] = two_body_integrals[1][p, q, r, s]

                        # Mixed spin in physicist 
                        two_body_coefficients[
                            2 * p, 2 * q + 1, 2 * r + 1, 2 * s
                        ] = two_body_integrals[2][p, q, r, s]
                        two_body_coefficients[
                            2 * p + 1, 2 * q, 2 * r, 2 * s + 1
                        ] = two_body_integrals[3][p, q, r, s]


        # Truncate.
        one_body_coefficients[np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.0
        two_body_coefficients[np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.0

        return one_body_coefficients, two_body_coefficients


In [94]:
from pathlib import Path
from nbed.driver import NbedDriver
import scipy as sp
from openfermion import get_sparse_operator

mol_filepath = Path("tests/molecules/water.xyz").absolute()

args = {
    "geometry": str(mol_filepath),
    "n_active_atoms": 1,
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-8,
    "savefile": None,
    "run_ccsd_emb": False,
    "run_fci_emb": False,
}

## Restricted calculation (using old method)

In [95]:
restric_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=False,
)

rfci = restric_driver._global_fci.e_tot - restric_driver._global_ks.energy_nuc()

builder = HamiltonianBuilder(restric_driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
rdiag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
rfci, rdiag[0], rfci - rdiag[0]


(-84.29484027700322, -84.29484027704268, 3.9463543544115964e-11)

### Explicit calculation of two body integrals

In [96]:
scf_method = restric_driver._global_ks

c_matrix_active = restric_driver._global_ks.mo_coeff

n_orbs = c_matrix_active.shape[1]

two_body_compressed = ao2mo.kernel(scf_method.mol, [c_matrix_active, c_matrix_active, c_matrix_active, c_matrix_active])

# get electron repulsion integrals
eri = ao2mo.restore(
    1, two_body_compressed, n_orbs
)  # no permutation symmetry

# Openfermion uses physicist notation whereas pyscf uses chemists
two_body_integrals = np.asarray(eri.transpose(0, 2, 3, 1), order="C")

In [97]:
two_body_compressed.shape

(28, 28)

## Force unrestricted

In [98]:
forced_driver = NbedDriver(
    geometry=args["geometry"],
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
)

ufci = forced_driver._global_fci.e_tot - unrestric_driver._global_ks.energy_nuc()
old_hcore = forced_driver._global_ks.get_hcore()
forced_driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])

builder = HamiltonianBuilder(forced_driver._global_ks, 0, "jordan_wigner")
ham = builder.build()
udiag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), k=1, which="SA")
ufci, udiag

(-84.29484027700319, array([-84.29484028]))

In [99]:
from nbed.driver import NbedDriver

unrestric_driver = NbedDriver(
    geometry=args["geometry"],
    charge=-1,
    spin=1,
    n_active_atoms=args["n_active_atoms"],
    basis=args["basis"],
    xc_functional=args["xc_functional"],
    projector=args["projector"],
    localization=args["localization"],
    convergence=args["convergence"],
    savefile=args["savefile"],
    run_ccsd_emb=args["run_ccsd_emb"],
    run_fci_emb=args["run_fci_emb"],
    force_unrestricted=True,
)

fci = unrestric_driver._global_fci.e_tot - unrestric_driver._global_ks.energy_nuc()
logger.info(f"FCI energy of unrestricted driver test: {fci}")

In [100]:
# Unrestricted FCI
fci

-83.68394377786774

In [87]:
# We need to double up the size of the hcore
old_hcore = unrestric_driver._global_ks.get_hcore()
unrestric_driver._global_ks.get_hcore = lambda *args: np.array([old_hcore, old_hcore])
builder = HamiltonianBuilder(unrestric_driver._global_ks, 0, "jordan_wigner")
ham = builder.build()

In [89]:
import scipy as sp
from openfermion import get_sparse_operator

diag, _ = sp.sparse.linalg.eigsh(get_sparse_operator(ham), which="SA")
diag

array([-84.29484028, -83.97614237, -83.97614237, -83.88821773,
       -83.88971265, -83.88971265])

## Run it all explicitly

things to check 

-[] one body integrals
-[] two body integrals
-[] spinorb from spatial

In [21]:
def _unrestricted_spinorb_from_spatial(one_body_integrals, two_body_integrals):
    n_qubits = one_body_integrals[0].shape[0] + one_body_integrals[1].shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[0, p, q]
            # one_body_coefficients[2 * p, 2 * q] = one_body_integrals[1, p, q]

            # one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[0, p, q]
            one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[1, p, q]

            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[
                        2 * p, 2 * q + 1, 2 * r + 1, 2 * s
                    ] = two_body_integrals[0][p, q, r, s]
                    two_body_coefficients[
                        2 * p + 1, 2 * q, 2 * r, 2 * s + 1
                    ] = two_body_integrals[1][p, q, r, s]

                    # Same spin
                    two_body_coefficients[
                        2 * p, 2 * q, 2 * r, 2 * s
                    ] = two_body_integrals[0][p, q, r, s]
                    two_body_coefficients[
                        2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1
                    ] = two_body_integrals[1][p, q, r, s]

    # Truncate.
    one_body_coefficients[np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.0
    two_body_coefficients[np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.0

    return one_body_coefficients, two_body_coefficients

In [27]:
unrestric_driver._global_ks.get_hcore()[0]

array([[-3.27321051e+01, -7.61545320e+00,  0.00000000e+00,
         1.08420217e-18,  1.91880201e-02, -1.79027902e+00,
        -1.79027902e+00],
       [-7.61545320e+00, -9.34493570e+00,  0.00000000e+00,
         0.00000000e+00,  2.24877651e-01, -3.80258461e+00,
        -3.80258461e+00],
       [ 0.00000000e+00,  0.00000000e+00, -7.46591598e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.08420217e-18,  0.00000000e+00,  0.00000000e+00,
        -7.62774933e+00, -3.46944695e-18, -2.06626916e+00,
         2.06626916e+00],
       [ 1.91880201e-02,  2.24877651e-01,  0.00000000e+00,
        -3.46944695e-18, -7.55970438e+00,  1.64669438e+00,
         1.64669438e+00],
       [-1.79027902e+00, -3.80258461e+00,  0.00000000e+00,
        -2.06626916e+00,  1.64669438e+00, -5.11488264e+00,
        -1.63403819e+00],
       [-1.79027902e+00, -3.80258461e+00,  0.00000000e+00,
         2.06626916e+00,  1.64669438e+00, -1.63403819e+00,
        -5.1148826

In [ ]:
qubit_reduction = 0

scf_method = unrestric_driver._global_ks

while True:

    c_matrix_active = scf_method.mo_coeff
    # one body terms
    one_body_integrals_alpha = (
        c_matrix_active[0].T
        @ scf_method.get_hcore()[0]
        @ c_matrix_active[0]
    )
    one_body_integrals_beta = (
        c_matrix_active[1].T
        @ scf_method.get_hcore()[1]
        @ c_matrix_active[1]
    )

    one_body_integrals = np.array(
        [one_body_integrals_alpha, one_body_integrals_beta]
    )

    c_matrix_active = scf_method.mo_coeff

    n_orbs_alpha = c_matrix_active[0].shape[1]
    n_orbs_beta = c_matrix_active[1].shape[1]

    two_body_compressed_alpha = ao2mo.kernel(
        scf_method.mol, c_matrix_active[0]
    )
    two_body_compressed_beta = ao2mo.kernel(
        scf_method.mol, c_matrix_active[1]
    )

    # get electron repulsion integrals
    eri_alpha = ao2mo.restore(
        1, two_body_compressed_alpha, n_orbs_alpha
    )  # no permutation symmetry
    eri_beta = ao2mo.restore(1, two_body_compressed_beta, n_orbs_beta)

    # Openfermion uses physicist notation whereas pyscf uses chemists
    two_body_integrals_alpha = np.asarray(
        eri_alpha.transpose(0, 2, 3, 1), order="C"
    )
    two_body_integrals_beta = np.asarray(
        eri_beta.transpose(0, 2, 3, 1), order="C"
    )

    two_body_integrals = np.array(
        [two_body_integrals_alpha, two_body_integrals_beta]
    )


    one_body_coefficients, two_body_coefficients = _unrestricted_spinorb_from_spatial(
        self._one_body_integrals, self._two_body_integrals
    )

    molecular_hamiltonian = InteractionOperator(
        (constant_e_shift + core_constant),
        one_body_coefficients,
        0.5 * two_body_coefficients,
    )

    qham = HamiltonianBuilder._qubit_transform(self.transform, molecular_hamiltonian)

    # Don't like this option sitting with the recursive
    # call beneath it - just a little too complicated.
    # ...but it works for now.
    if taper is True:
        qham = self._taper(qham)
    if n_qubits is None:
        logger.debug("Unreduced Hamiltonain found.")
        return qham

    # Wanted to do a recursive thing to get the correct number
    # from tapering but it takes ages.
    final_n_qubits = count_qubits(qham)
    if final_n_qubits <= n_qubits:
        logger.debug("Hamiltonian reduced to %s qubits.", final_n_qubits)
        return qham

    # Check that we have the right number of qubits.
    qubit_reduction += final_n_qubits - n_qubits

## Options & Questions

- Can we test by splitting/doubling up the closed shell hamiltonian?
- How do we calculate the 1e and 2e terms for restricted/unrestricted?
